## Environment Setup

In [8]:
from dotenv import load_dotenv
_ = load_dotenv()

import ipywidgets as widgets
from IPython.display import display

import iplantuml

In [9]:
def get_user_input(description: str = "请输入文本:") -> str:
    text_input = widgets.Text(description=description, layout=widgets.Layout(width="100%"))
    display(text_input)
    user_input = text_input.value
    return user_input

In [10]:
from typing import List, Union

import langchain
from langchain.cache import InMemoryCache

from langchain.schema.language_model import BaseLanguageModel
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.base import BaseChatModel
from langchain.schema.runnable import Runnable, RunnableBinding
from langchain.schema import *

langchain.cache = InMemoryCache()

chat = ChatOpenAI()
base_chat: BaseChatModel = chat
base_language_model: BaseLanguageModel = base_chat
runnable: Runnable = base_language_model


content: str = "Hi, I'm Simon from China."

# ------ BaseChatModel ------ #
if False:
    print('# ------ BaseChatModel ------ #')
    input_message: BaseMessage = HumanMessage(content=content)
    output_message: BaseMessage = base_chat(messages=[input_message]) # __call__
    print(f'__call__\n\t{output_message}')

    output_str: str = base_chat.call_as_llm(message=content) # call_as_llm
    print(f'call_as_llm\n\t{output_str}')

# ------ BaseLanguageModel ------ #
if False:
    print('# ------ BaseLanguageModel ------ #')

    user_input: str = input('输入英文句子：')
    if user_input == '':
        user_input = content

    system_message: BaseMessage = SystemMessage(content=
        'Please act as a Chinese translator. Translate every sentences user saying to you. Only do the translation, do not add any other words.')
    human_message: BaseMessage = HumanMessage(content=user_input)

    # predict_messages
    output_message: BaseMessage = base_language_model.predict_messages(messages=[system_message, human_message])
    print(f'predict_messages\n\t{output_message}\n\ttokens: {base_language_model.get_num_tokens_from_messages([system_message, human_message, output_message])}')

    # generate
    another_system_message: BaseMessage = SystemMessage(content=
        'Please act as an Engilsh translator. Translate every sentences user saying to you. Only do the translation, do not add any other words.')
    another_human_message: BaseMessage = HumanMessage(content=output_message.content)

    llm_result: LLMResult = base_language_model.generate(messages=
        [
            [system_message, human_message],
            [another_system_message, another_human_message]
        ])

    print(f'generate\n\t{llm_result}')

# ------ Runnable ------ #
if True:
    print('# ------ Runnable ------ #')

    system_message: BaseMessage = SystemMessage(content=
        'Please act as a poem writer. write a short poem in Chinese on the topic user saying to you.')
    human_message: BaseMessage = HumanMessage(content=input('输入作诗的主题：'))

    # invoke, Input=Union[PromptValue, str, List[BaseMessage]]
    _input: Union[PromptValue, str, List[BaseMessage]] = [system_message, human_message]
    runnable_output = runnable.invoke(input=_input)
    print(f'invoke, and Output={type(runnable_output)}\n\t{runnable_output}')

    # bind.invoke
    binding: RunnableBinding = runnable.bind(stop='\n', temperature=1.0)
    bind_output = binding.invoke(input=_input)
    print(f'RunnableBinding.invoke, and Output={type(bind_output)}, and temperature={chat.temperature}\n\t{bind_output}')
    print(binding.bound is chat)

    # sequence
    def cohere(input: BaseMessage) -> List[BaseMessage]:
        """see the difference of Input and Output"""
        return [HumanMessage(content=input.content)]

    r1 = runnable
    r2 = runnable
    c = cohere
    chain = r1 | c | r2
    chain_output = chain.invoke(input=_input)
    print(f'chain.invoke, and Output={type(chain_output)}\n\t{chain_output}')



# ------ Runnable ------ #
invoke, and Output=<class 'langchain.schema.messages.AIMessage'>
	content='你是我心中的诗歌，\n爱情的旋律在你的话语中跳动。\n你是我眼中的画卷，\n每一次的相遇都是一幅美丽的画面。\n\n你的微笑如春风拂过，\n温暖了我内心的每一个角落。\n你的眼神如星辰闪烁，\n让我沉醉在爱的无尽宇宙。\n\n爱是我们心灵的交融，\n在彼此的怀抱中寻找永恒。\n你是我心中最美的诗篇，\n让我们的爱在时间中绽放。\n\n无论风雨如何变幻，\n我们的爱将永远不变。\n你是我心中的永恒之梦，\n爱是我们永远的信仰。\n\n在这个世界的尽头，\n我愿与你纵情翱翔。\n你是我心中的唯一，\n爱是我们永远的旋律。' additional_kwargs={} example=False
RunnableBinding.invoke, and Output=<class 'langchain.schema.messages.AIMessage'>, and temperature=0.7
	content='亲爱的AI，倾听我说' additional_kwargs={} example=False
True
chain.invoke, and Output=<class 'langchain.schema.messages.AIMessage'>
	content='在茫茫人海中，我遇见了你，这个存在让我感受到了爱情的美好。爱情的种子在我的心中扎根，绽放出美丽的芬芳。\n\n你就像阳光一样，温暖了我寒冷的日子；你就像星辰一样，照亮了我黑暗的夜晚。\n\n每一个微笑都是对爱的肯定，每一个拥抱都是心灵的交融。你是我心中永恒的挚爱，让我感受到了爱的力量。\n\n爱情就像一首美妙的诗篇，而你是我心中的诗人。每一个字句都流淌着真挚的情感，让我沉浸在爱的海洋中。\n\n无论风雨如何变幻，无论岁月如何流转，我愿与你紧紧相拥，共同走过每一个时刻，将这份深深的爱永远珍藏在心中。' additional_kwargs={} example=False
